In [1]:
import warnings
warnings.filterwarnings("ignore")
from google.cloud import bigquery
import pandas as pd
import uuid
import logging

In [2]:
def state_search(state=None):
    # State dictionary for state codes
    state_dict = {'AL': 1, 'AK': 2, 'AZ': 3, 'AR': 4, 'CA': 5, 'CO': 6, 'CT': 7,
                  'DE': 8, 'DC': 9, 'FL': 10, 'GA': 11, 'HI': 12, 'ID': 13, 'IL': 14,
                  'IN': 15, 'IA': 16, 'KS': 17, 'KY': 18, 'LA': 19, 'ME': 20, 'MD': 21,
                  'MA': 22, 'MI': 23, 'MN': 24, 'MS': 25, 'MO': 26, 'MT': 27, 'NE': 28,
                  'NV': 29, 'NH': 30, 'NJ': 31, 'NM': 32, 'NY': 33, 'NC': 34, 'ND': 35,
                  'OH': 36, 'OK': 37, 'OR': 38, 'PA': 39, 'RI': 41, 'SC': 42, 'SD': 43,
                  'TN': 44, 'TX': 45, 'UT': 46, 'VT': 47, 'VA': 49, 'WA': 50, 'WV': 51,
                  'WI': 52, 'WY': 53, 'OTHER': 54}
    
    try:
        state_code = state_dict[state.upper()]
    except:
        state_code = 54
    return state_code

In [3]:
def input_data(age, gender, race=None, state=None, alzheimers=None, heart_failure=None, 
               kidney_disease=None, cancer=None, copd=None, depression=None, 
               diabetes=None, heart_disease=None, osteoporosis=None, arthritis=None, 
               stroke=None, dx=None, px=None, hcpcs=None):
    
    # Connect to database
    database = 'healthcare-predictions'
    client = bigquery.Client(database)
    
    # Raise exceptions
    if type(age) is not int:
        raise Exception('Age must be a number')
        logging.error('Age must be a number')
        return
    
    if gender not in (1, 2):
        raise Exception('Gender must be 1 or 2')
        logging.error('Gender must be 1 or 2')
        return
    
    # Lookup state code
    state_code = state_search(state)
    
    # Impute none values
    if race not in (1, 2, 5):
        race = 3
    if alzheimers is None:
        alzheimers = 2
    elif alzheimers not in (None, 1, 2):
        raise Exception('Alzheimers must be 1 or 2')
        logging.error('Alzheimers must be 1 or 2')
        return
    if heart_failure is None:
        heart_failure = 2
    elif heart_failure not in (1, 2):
        raise Exception('Heart failure must be 1 or 2')
        logging.error('Heart failure must be 1 or 2')
        return
    if kidney_disease is None:
        kidney_disease = 2
    elif kidney_disease not in (1, 2):
        raise Exception('Kidney disease must be 1 or 2')
        logging.error('Kidney disease must be 1 or 2')
        return
    if cancer is None:
        cancer = 2
    elif cancer not in (1, 2):
        raise Exception('Cancer must be 1 or 2')
        logging.error('Cancer must be 1 or 2')
        return
    if copd is None:
        copd = 2
    elif copd not in (1, 2):
        raise Exception('COPD must be 1 or 2')
        logging.error('COPD must be 1 or 2')
        return
    if depression is None:
        depression = 2
    elif depression not in (1, 2):
        raise Exception('Depression must be 1 or 2')
        logging.error('Depression must be 1 or 2')
        return
    if diabetes is None:
        diabetes = 2
    elif diabetes not in (1, 2):
        raise Exception('Diabetes must be 1 or 2')
        logging.error('Diabetes must be 1 or 2')
        return
    if heart_disease is None:
        heart_disease = 2
    elif heart_disease not in (1, 2):
        raise Exception('Heart disease must be 1 or 2')
        logging.error('Heart disease must be 1 or 2')
        return
    if osteoporosis is None:
        osteoporosis = 2
    elif osteoporosis not in (1, 2):
        raise Exception('Osteoporosis must be 1 or 2')
        logging.error('Osteoporosis must be 1 or 2')
        return
    if arthritis is None:
        arthritis = 2
    elif arthritis not in (1, 2):
        raise Exception('Arthritis must be 1 or 2')
        logging.error('Arthritis must be 1 or 2')
        return
    if stroke is None:
        stroke = 2
    elif stroke not in (1, 2):
        raise Exception('Stroke must be 1 or 2')
        logging.error('Stroke must be 1 or 2')
        return
    if dx is None:
        dx = 0
    elif type(dx) is not int:
        raise Exception('Number of diagnosis must be a number')
        logging.error('Number of diagnosis must be a number')
        return
    if px is None:
        px = 0
    elif type(px) is not int:
        raise Exception('Number of procedures must be a number')
        logging.error('Number of procedures must be a number')
        return
    if hcpcs is None:
        hcpcs = 0
    elif type(hcpcs) is not int:
        raise Exception('Number of services outside of primary insurance must be a number')
        logging.error('Number of services outside of primary insurance must be a number')
        return
    
    
    # Assign unique id
    id = str(uuid.uuid4())
    
    #Insert data to table
    query = """
    INSERT INTO `cms.predictions`
    VALUES(""" \
    + '\''+ id + '\'' + ',' \
    + str(age) + ',' \
    + str(gender) + ',' \
    + str(race) + ',' \
    + str(state_code) + ',' \
    + str(alzheimers) + ',' \
    + str(heart_failure) + ',' \
    + str(kidney_disease) + ',' \
    + str(cancer) + ',' \
    + str(copd) + ',' \
    + str(depression) + ',' \
    + str(diabetes) + ',' \
    + str(heart_disease) + ',' \
    + str(osteoporosis) + ',' \
    + str(arthritis) + ',' \
    + str(stroke) + ',' \
    + str(dx) + ',' \
    + str(px) + ',' \
    + str(hcpcs) + ')'
    
    # Run query
    query_job = client.query(query)
    query_job.result()
    job_id = query_job.job_id
    if query_job.state == 'DONE':
        print('Insert Job ID: ' + job_id + ' is ' + query_job.state)
        logging.info('Insert Job ID: ' + job_id + ' is ' + query_job.state)
    else:
        raise Exception('Insert Job ID: ' + job_id + ' error ' + query_job.errors)
        print('Insert Job ID: ' + job_id + ' error ' + query_job.errors)
        logging.error('Insert Job ID: ' + job_id + ' error ' + query_job.errors)
        return
    return id    

In [14]:
def prediction(age, gender, race=None, state=None, alzheimers=None, heart_failure=None, 
               kidney_disease=None, cancer=None, copd=None, depression=None, 
               diabetes=None, heart_disease=None, osteoporosis=None, arthritis=None, 
               stroke=None, dx=None, px=None, hcpcs=None):
    
    # Connect to database
    database = 'healthcare-predictions'
    client = bigquery.Client(database)
    
    try:
        # Input data to table
        id = input_data(age, gender, race, state, alzheimers, heart_failure, 
                        kidney_disease, cancer, copd, depression, 
                        diabetes, heart_disease, osteoporosis, arthritis, 
                        stroke, dx, px, hcpcs)
        
        # Prediction query
        query = """
        SELECT *
        FROM
            ML.PREDICT(MODEL `cms.model_v1`,
                (
                SELECT *
                FROM 
                    `cms.predictions`
                WHERE ID = """ '\'' + id + '\'' \
                """
                )
            )"""
        
        # Run query
        query_job = client.query(query)
        results = query_job.result()
        job_id = query_job.job_id
        if query_job.state == 'DONE':
            print('Prediction Job ID: ' + job_id + ' is ' + query_job.state)
            logging.info('Prediction Job ID: ' + job_id + ' is ' + query_job.state)
        else:
            raise Exception('Insert Job ID: ' + job_id + ' error ' + query_job.errors)
            print('Prediction Job ID: ' + job_id + ' error ' + query_job.errors)
            logging.error('Prediction Job ID: ' + job_id + ' error ' + query_job.errors)
            return
        
        # Get predicted annual cost 
        for row in results:
            prediction = {'prediction': row.predicted_ANNUAL_COST}
            logging.info(prediction)
            print(prediction)
        return prediction
    
    except Exception as e:
        logging.error(e)
        #error = {'error': e}
        error = e.__dict__
        #print(error)
        return error

In [6]:
import json

In [15]:
#prediction(50, 3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1)

ERROR:root:Gender must be 1 or 2


{}